<a href="https://colab.research.google.com/github/SilvioIA2025/ALura2025/blob/main/Projeto_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [229]:
%pip -q install google-genai

In [230]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [233]:
# Configura o cliente da SDK do Gemini

from google import genai
client = genai.Client()
from IPython.display import HTML, Markdown



In [234]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [235]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [262]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
import textwrap
from rich.markdown import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


In [263]:
##################################################################
# --- Agente 1: Buscador de Custos de Viagens Internacionais --- #
##################################################################
def agente_buscador(topico, pais, perfil, data_de_inicio, data_de_fim, numero_adultos, numero_crianças):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Como um especialista em calcular gastos em viagens internacionais, meu objetivo é fornecer uma estimativa detalhada e otimizada dos seus custos,
        considerando suas preferências e buscando as melhores opções em cada categoria.

        Instruções para o Especialista em Cálculos de Viagem:

        1. Compreensão do Perfil do Viajante:

        Destino: País(es) e Cidade(s).
        Período: Datas de partida e retorno (para considerar alta/baixa temporada e eventos).
        Número de Viajantes: Adultos e crianças.
        Estilo de Viagem: (e.g., mochileiro, econômico, confortável, luxuoso) para calibrar os valores.
        Interesses Específicos: (e.g., gastronomia, museus, aventura, compras) para alocar orçamento.

        2. Análise e Estimativa por Categoria:

        Voos (Melhores Opções):
        Pesquisar e comparar companhias aéreas, escalas, duração do voo e aeroportos de partida/chegada.
        Considerar voos diretos vs. com escalas (custo/benefício).
        Avaliar épocas de compra (antecedência) e flexibilidade de datas para melhores preços.
        Incluir custos de bagagem despachada, se aplicável.

        Aluguel de Carro:
        Avaliar a necessidade do aluguel versus transporte público no destino.
        Selecionar o tipo de veículo adequado (econômico, SUV, etc.).
        Incluir custos de seguro obrigatório e opcionais, combustível, pedágios e estacionamento.
        Comparar locadoras e políticas de retirada/devolução.

        Hotéis/Hospedagem (Melhores Opções):
        Sugerir tipos de hospedagem (hotéis, hostels, Airbnb, pousadas) com base no estilo de viagem.
        Considerar localização (proximidade a atrações, transporte), avaliações, comodidades (café da manhã, Wi-Fi) e política de cancelamento.
        Estimar custo por noite, ajustado pela temporada.

        Refeições:
        Estimar um custo médio diário por pessoa para café da manhã, almoço e jantar, diferenciando entre restaurantes econômicos, médios e sofisticados.
        Considerar o custo de lanches, bebidas e gorjetas.
        Perguntar se o viajante pretende cozinhar algumas refeições.

        Atrações Turísticas e Atividades:
        Listar as principais atrações pagas e o custo de seus ingressos.
        Sugerir passes turísticos (se vantajosos para a quantidade de atrações).
        Incluir custo de tours guiados, shows ou atividades específicas desejadas.

        Transporte Local:
        Custos de metrô, ônibus, bondes, táxis ou aplicativos de transporte (Uber, Lyft, etc.).
        Estimar passes diários/semanais se forem mais econômicos.
        Despesas Gerais e Extras:

        Seguro Viagem: Item obrigatório e essencial.
        Visto e Taxas de Entrada: Se aplicável ao destino.
        Comunicações: Custo de chip local ou roaming internacional.
        Emergências/Imprevistos: Sugerir uma reserva de 10-15% do total para gastos não planejados.
        Compras/Souvenirs: Definir um orçamento para compras pessoais e lembrancinhas.
        Outros: Lavanderia, gorjetas adicionais, pequenos gastos diversos.

        3. Saída Final:
        Faça voce a escolha de uma cidade a ser visitada no pais, caso o não tenha sido escolhida, retorne todos os resultados de custo pelo
        estilo de Viagem selecionado no perfil (mochileiro, economico, confortavel e luxuoso) e voce deve escolher automaticamente dois
        Interesses Específicos: dentre Gastronomia, Museus, Aventura e Compras.
        Apresentar um orçamento detalhado por categoria (Voos, Aluguel de Carro, Hotéis/Hospedagem, Refeições, Atrações Turísticas e Atividades, Transporte Local, Seguro Viagem
        Visto e Taxas de Entrada, Comunicações, Emergências/Imprevistos, Compras/Souvenirs e Outros ) , com uma estimativa total convertida para a moeda de origem do viajante
        (geralmente BRL) e da moeda local do destino, utilizando a taxa de câmbio atualizada. Oferecer dicas de economia e personalização.

        A sua tarefa é usar a ferramenta de busca do google (google_search) respeitando a data inicial e final da viagem
        Foque em no maximo 4 opções relevantes respeitando os estilos de Viagem: (mochileiro, econômico, confortável, luxuoso), com base no preço, categoria, e avaliaçoes dos clientes anteriores.
        Se uma avaliação tiver poucas informações ou reações entusiasmadas, é possivel que ela não seja tão relevante assim
        e pode ser substituido por outro que tenha mais.
        Essas avaliações relevantes devem ser atuais, de no maximo um tres meses da data de hoje.
        """,
        description = "Agente que busca melhores voos, carros, hoteis, refeiçoes, atrações turisticas, transporte e custos diretos e indiretos de viagem indicado no Google",
        tools=[google_search],
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nPaís: {pais}\nPerfil: {perfil}\nData de inicio: {data_de_inicio}\nData de fim: {data_de_fim}\nNumero_adultos: {numero_adultos}\nNumero_crianças: {numero_crianças}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [264]:
##########################################
# --- Agente 2: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, lancamentos_buscados):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em documentos profissionais.
            Por ter um público amplo, entre 18 e 70 anos, use sempre um tom de escrita adequado.
            Revise o lancamentos_buscados abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Separar por uma linha em branco cada item da lista.
            Se o lancamentos_buscados estiver bom, repita apenas o texto novamente'.
            Caso haja problemas, corrija e refaça o texto de maneira adequada, gerando uma versão corrigida
            """,
        description="Agente revisor de post para documentos profissionais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {lancamentos_buscados}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [265]:
##################################################################
# --- Agente 3: Buscador da Melhor alternativa de levar ME --- #
##################################################################
def agente_moeda(topico, pais):
    imagem = Agent(
        name="agente_moeda",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Voce como especialista em cambio da Viktap, por favor me faça um breve resumo de qual a melhor maneira para eu converter o meu
        dinheiro local para o dinheiro internacional, considerando cartao de credito, cartao de debito, cartao internacional, remessa para conta
        no estrangeiro, ou PIX. No caso do PIX, fale sobre o viktap consultando a fonte no site https://viktap.com.br/
        """,
        tools=[google_search],
    )
    entrada_do_agente_moeda = f"Tópico: {topico}\nPais: {pais}"
    # Executa o agente
    escolha_moeda = call_agent(imagem, entrada_do_agente_moeda)
    return escolha_moeda

In [266]:
####################################################################
# --- Agente 4: Sugestao de Voo, pelo perfil escolhido ----------- #
####################################################################
def agente_viagem(topico, pais, perfil, aeroporto, data_inicio, data_retorno):
    viagem = Agent(
        name="agente_viagem",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Como um especialista em viagens internacionais da LATAM, faça apenas uma opção resumida para um voo consultando a LATAM saindo
        do campo aeroporto e na data de ida data_inicio e data de retorno data_retorno
        por favor escolha um voo direto de ida e um voo direto de volta a sua escolha informando o numero do voo e horario de partida e chegada
        """,
        tools=[google_search],
    )
    entrada_do_agente_viagem = f"Tópico: {topico}\nPais: {pais}\nPerfil: {perfil}\nAeroporto: {aeroporto}\nInicio: {data_inicio}\nFim: {data_retorno}"
    # Executa o agente
    viagem_pais = call_agent(viagem, entrada_do_agente_viagem)
    return viagem_pais

In [267]:
################################################################
# --- Agente 5: Sugestao de Hotel, refeição e passeio turistico
#     pelo perfil escolhido ---------------------------------- #
################################################################
def agente_outros(topico, pais, perfil, data_inicio, data_retorno):
    outros = Agent(
        name="agente_outros",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Como um especialista em viagens internacionais, faça apenas uma opção resumida para hoteis, refeição e passeios turisticos
        levando em consideração na data de ida data_inicio e data de retorno data_retorno
        por favor escolha somente 3 opções de hoteis, passeios turisticos e refeicoes.
        """,
        tools=[google_search],
    )
    entrada_do_agente_outros = f"Tópico: {topico}\nPais: {pais}\nPerfil: {perfil}\nInicio: {data_inicio}\nFim: {data_retorno}"
    # Executa o agente
    outros = call_agent(outros, entrada_do_agente_outros)
    return outros

In [268]:
##agente_buscador(topico, pais, data_inicio, data_retorno, numero_adultos, numero_crianças)
##agente_revisor(topico, lancamentos_buscados)
##agente_moeda(topico, pais)
##agente_viagem(topico, pais, perfil, aeroporto, data_inicio, data_retorno)
##agente_outros(topico, pais, perfil, data_inicio, data_retorno)

from datetime import datetime

def obter_data_do_usuario(param1):
    while True:
        if param1 == 'ida':
          data_str = input("Digite a data de inicio (DD/MM/AAAA): ")
        else:
          data_str = input("Digite a data de retorno (DD/MM/AAAA): ")

        try:
            # Tenta converter a string para um objeto datetime
            data_obj = datetime.strptime(data_str, "%d/%m/%Y")
            return data_obj
        except ValueError:
            print("Formato de data inválido. Por favor, use DD/MM/AAAA.")

def selecionar_perfil(opcoes, mensagem="Selecione uma opção:"):
    print(mensagem)
    for i, opcao in enumerate(opcoes):
        print(f"[{i + 1}] {opcao}")

    while True:
        try:
            escolha = input("Digite o número da sua escolha: ")
            indice_escolhido = int(escolha) - 1 # Ajusta para o índice da lista (0-based)

            if 0 <= indice_escolhido < len(opcoes):
                return opcoes[indice_escolhido]
            else:
                print("Número inválido. Por favor, digite um número da lista.")
        except ValueError:
            print("Entrada inválida. Por favor, digite um número.")



In [281]:
topico = 'Viagem'
pais = input("Digite o Pais/Cidade do destino: ")
data_inicio = obter_data_do_usuario('ida')
data_retorno = obter_data_do_usuario('fim')
adultos = input("Digite o numero de adultos: ")
criancas = input("Digite o numero de crianças: ")
opcoes = ["Mochileiro", "Economico", "Confortavel", "Luxuoso"]
perfil = selecionar_perfil(opcoes, mensagem="Selecione uma opção:")
print (perfil)
aeroporto = input("Digite o aeroporto de partida: ")

lancamentos_buscados = agente_buscador(topico, pais, perfil, data_inicio, data_retorno, adultos, criancas)
##print("\n Resultado do Agente 1 (Buscador)! \n")
##display(to_markdown(lancamentos_buscados))
##print("----------------------------------------------------------------")

post_final = agente_revisor(topico, lancamentos_buscados)
display(to_markdown("-------------------------------"))
display(to_markdown("Recomendações para a sua viagem"))
display(to_markdown("-------------------------------"))
display(to_markdown(post_final))

escolha_moeda = agente_moeda(topico, pais)
display(to_markdown("--------------------------------------------------"))
display(to_markdown("Dicas financeiras para pagar com moeda estrangeira"))
display(to_markdown("--------------------------------------------------"))
display(to_markdown(escolha_moeda))

viagem_pais = agente_viagem(topico, pais, perfil, aeroporto, data_inicio, data_retorno)
display(to_markdown("------------------------------------"))
display(to_markdown("Dica de um agente de viagem da LATAM"))
display(to_markdown("------------------------------------"))
display(to_markdown(viagem_pais))

outros = agente_outros(topico, pais, perfil, data_inicio, data_retorno)
display(to_markdown("---------------------------------------------------------"))
display(to_markdown("Algumas dicas de Hoteis, refeiçoes e passeios turisticos"))
display(to_markdown("---------------------------------------------------------"))
display(to_markdown(outros))


Digite o Pais/Cidade do destino: Canada
Digite a data de inicio (DD/MM/AAAA): 15/06/2025
Digite a data de retorno (DD/MM/AAAA): 30/06/2025
Digite o numero de adultos: 1
Digite o numero de crianças: 0
Selecione uma opção:
[1] Mochileiro
[2] Economico
[3] Confortavel
[4] Luxuoso
Digite o número da sua escolha: 1
Mochileiro
Digite o aeroporto de partida: gru


▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

▌ Recomendações para a sua viagem                                                                                

▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

▌ Olá! Como seu especialista em calcular gastos de viagem, vou te ajudar a planejar essa aventura de mochileiro  
▌ pelo Canadá. Considerando o seu perfil de mochileiro, a duração da viagem de 15 de junho a 30 de junho de 2025 
▌ e o fato de ser uma viagem individual, escolhi Montreal como destino dentro do Canadá. Montreal oferece uma    
▌ excelente infraestrutura para mochileiros, com boa rede de transporte público, hostels e muitas atividades     
▌ culturais e gastronômicas acessíveis.                                                                          
▌ Para complementar o seu estilo de viagem, selecionei dois interesses específicos que combinam com o perfil     
▌ mochileiro em Montreal: Gastronomia (explorar a culinária local de forma econômica, como mercados e            
▌ lanchonetes) e Museus (aproveitar a rica cena cultural da cidade, buscando opções gratuitas ou com preços      
▌ acessíveis).                                                                                                   
▌ Agora, vou realizar as buscas para estimar os custos por categoria para a sua viagem a Montreal.               
▌ Okay, com base nas buscas e no seu perfil de mochileiro interessado em gastronomia e museus, preparei a        
▌ seguinte estimativa de gastos para sua viagem de 15 dias em Montreal, de 15 a 30 de junho de 2025:             
▌ Cotação Atual (Maio de 2025): 1 CAD ≈ 4,06 BRL                                                                 
▌ Estimativa de Gastos para 1 pessoa (Mochileiro) - 15 dias em Montreal                                          
▌  • Voos (ida e volta para São Paulo - Montreal):                                                               
▌     • Pesquisas indicam voos de ida e volta de São Paulo (GRU) para Montreal (YUL) em junho de 2025 a partir de
▌       R$ 3.204, podendo chegar a valores bem mais altos dependendo da companhia e escalas.                     
▌     • Considerando o perfil mochileiro, buscamos a opção mais econômica, que geralmente envolve voos com       
▌       escalas.                                                                                                 
▌     • Estimativa Mochileiro: R$ 3.500 - R$ 5.000 (aproximadamente 860 CAD - 1230 CAD) - Estimaremos R$ 4.250   
▌       (aproximadamente 1047 CAD)                                                                               
▌  • Aluguel de Carro:                                                                                           
▌     • Para Montreal, o transporte público é eficiente e amplamente utilizado por turistas, sendo uma opção mais
▌       econômica para mochileiros. Alugar carro é mais recomendado para visitar cidades vizinhas.               
▌     • Estimativa Mochileiro: R$ 0                                                                              
▌  • Hotéis/Hospedagem (15 noites):                                                                              
▌     • Hostels são a opção ideal para mochileiros, oferecendo bom custo-benefício e oportunidade de socializar. 
▌     • A diária em hostels em Montreal varia, mas é possível encontrar opções a partir de R$ 108.               
▌     • Estimativa Mochileiro: R$ 108/noite * 15 noites = R$ 1.620 (aproximadamente 399 CAD) - Estimaremos R$    
▌       1.800 (aproximadamente 443 CAD) para ter uma pequena margem.                                             
▌  • Refeições (15 dias):                                                                                        
▌     • Com foco em gastronomia econômica, a ideia é explorar mercados, lanchonetes e cozinhar algumas refeições 
▌       no hostel (se disponível).                                                                               
▌     • O custo médio de uma refeição barata em Montreal é de cerca de C$ 14,30. É possível encontrar opções por 
▌       menos de 10 dólares, como bagels e sanduíches.                                                           
▌     • Es

...

▌ Como especialista em viagens internacionais da LATAM, buscando um voo direto de ida e volta para o Canadá      
▌ saindo de Guarulhos (GRU) entre 15/06/2025 e 30/06/2025, apresento a seguinte opção resumida com a LATAM:      
▌ Atualmente, a LATAM não opera voos diretos de São Paulo (GRU) para o Canadá. As rotas da LATAM para o Canadá a 
▌ partir de GRU geralmente incluem pelo menos uma escala.                                                        
▌ Para a sua viagem entre 15/06/2025 e 30/06/2025, a LATAM opera voos para o Canadá com conexões, principalmente 
▌ para destinos como Toronto (YYZ) e Montreal (YUL).                                                             
▌ Para encontrar a melhor opção com a LATAM para as suas datas, sugiro que consulte diretamente o site da LATAM  
▌ ou entre em contato com a central de atendimento para verificar as rotas com o menor número de escalas e os    
▌ horários disponíveis para o período desejado.                                                                  

▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

▌ Algumas dicas de Hoteis, refeiçoes e passeios turisticos                                                       

▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

▌ Para sua viagem de mochileiro ao Canadá entre 15/06/2025 e 30/06/2025, focando em custo-benefício, aqui estão 3
▌ opções resumidas de hotéis (hostels), refeições e passeios turísticos:                                         
▌ Hospedagem (Hostels):                                                                                          
▌  1 Samesun Banff: Frequentemente classificado como um dos melhores hostels em Banff para mochileiros, oferece  
▌    dormitórios e cozinha compartilhada para economizar com refeições. Sua localização central facilita o acesso
▌    a muitas atrações.                                                                                          
▌  2 HI Banff Alpine Centre: Uma ótima opção para viajantes individuais, possui cozinha equipada, lounge e       
▌    espaços de co-working.                                                                                      
▌  3 HI Vancouver Downtown: Localizado perto da English Bay Beach e do Stanley Park, este hostel oferece cozinha 
▌    compartilhada e é ideal para quem busca explorar o centro de Vancouver a pé.                                
▌ Refeições (Custo-Benefício):                                                                                   
▌  1 Mercados e Mercearias: Comprar em mercados locais e preparar suas próprias refeições nos hostels é a maneira
▌    mais econômica de comer. Em Vancouver, o Granville Island Market oferece uma variedade de ingredientes. Em  
▌    Banff, considere comprar em Canmore para uma seleção maior.                                                 
▌  2 Comida de Rua e Lanchonetes: Procure por opções de comida de rua e lanchonetes para refeições rápidas e     
▌    baratas. Em Banff, estabelecimentos como BARBurrito, Dominos, Subway, McDonalds, Banff Shawarma e Hankki são
▌    mencionados como opções mais acessíveis. Em Vancouver, a comida de rua e restaurantes chineses, incluindo   
▌    dim sum, são recomendados.                                                                                  
▌  3 Pubs com Especiais: Alguns pubs podem oferecer promoções de refeições com bom custo-benefício. O Beaver, no 
▌    Samesun Hostel em Banff, já foi conhecido por ter especiais de refeição acessíveis.                         
▌ Passeios Turísticos (Foco em Atividades Gratuitas/Baratas):                                                    
▌  1 Explorar Parques Nacionais (Banff): Aproveite as diversas atividades gratuitas e de baixo custo em Banff    
▌    National Park. Caminhadas em trilhas como Tunnel Mountain, Fenlands Trail e Bow River Loop são gratuitas.   
▌    Visitar Bow Falls, Cascade Gardens e o Cave and Basin (entrada gratuita com o Parks Pass) também são opções.
▌    O acesso ao Banff National Park tem uma taxa diária ou você pode comprar um Discovery Pass anual, que pode  
▌    ser mais vantajoso dependendo do seu roteiro. A entrada para jovens até 17 anos é gratuita em locais        
▌    administrados pela Parks Canada.                                                                            
▌  2 Aproveitar as Áreas Naturais em Vancouver: Explore parques como o Stanley Park (entrada gratuita) e caminhe 
▌    pela orla (Seawall). Visitar praias como Kitsilano Beach e English Bay também são atividades de baixo custo.
▌    O Lynn Canyon Park oferece trilhas e uma ponte suspensa gratuita.                                           
▌  3 Descobrir a Cultura Local (Vancouver e Banff): Em Vancouver, explore bairros como Gastown e Chinatown a pé. 
▌    Visite mercados de fazendeiros que geralmente têm entrada gratuita. Em Banff, algumas galerias de arte e    
▌    museus têm entrada gratuita ou taxas acessíveis. Verifique a programação local para eventos gratuitos em    
▌    junho.                                                                                                      